# Tool Use with SambaCloud 

In this example, we’ll demonstrate how to use tool use with models from SambaCloud. We’ll integrate a simple function called get_time, which the model will be able to call when appropriate.

## Initial Setup

Import the necessary libraries

In [1]:
#!pip install sambanova

In [2]:
import sambanova
import json
from datetime import datetime
import os


Define the model

In [3]:
#Initialize the OpenAI client using the SambaCloud base URL and your generated API key from the portal
client = sambanova.SambaNova(
    base_url="https://api.sambanova.ai/v1",
    api_key=os.getenv("SAMBANOVA_API_KEY") # Enter your API key here
)

# Specify the model
MODEL = 'Meta-Llama-3.3-70B-Instruct' #Meta-Llama-3.3-70B-Instruct, gpt-oss-120b

## Step 1: Define the tools

Define the function(s)/tool(s) that the LLM will use. In this example, we're creating a simple function to get the current date, current time or both. 

In [4]:
def get_time(kind: str = 'both') -> str:
    """Returns current date, current time or both.
    Args:
        kind(str): date, time or both
    
    Returns:
        str: current date, current time or both    
    """
    if kind == "date":
        date = datetime.now().strftime("%m/%d/%Y")
        return f"Current date: {date}"
    elif kind == "time":
        time = datetime.now().strftime("%H:%M:%S")
        return f"Current time: {time}"
    else:
        date = datetime.now().strftime("%m/%d/%Y")
        time = datetime.now().strftime("%H:%M:%S")
        return f"Current date: {date}, Current time: {time}"

## Step 2: Define the tool schemas

Define a JSON schema for each function/tool. You will need to specify:
- The name of the function and a description of what it does.
- The input parameters, their data types, and their descriptions.

In [5]:
# List of tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "Returns current date, current time or both",
            "parameters": {
                "type": "object",
                "properties": {
                    "kind": {
                        "type": "string",
                        "description": "kind of information to retrieve \"date\", \"time\" or \"both\"",
                    }
                },
                "required": ["kind"]
            },
        }
    },
]

## Step 3: Configure tool use in your API call

When sending an API request to SN Cloud, include the tool schemas in the tools parameter

In [6]:
# Specify messages
messages = [
    {"role": "system", "content": "You are a helpful assistant with access to the get_time tool. Use the tool to get the time or date when needed, otherwise responds conversationally."},
    {"role": "user", "content": "What is the time and date now?"}
]

# Make API call
completion = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools
)

print(completion.choices[0].message)

ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_2799bf3648024ecc9f', function=ChoiceMessageToolCallFunction(arguments='{"kind":"both"}', name='get_time'), type='function', index=None)])


## Step 4: Handle tool calls

If the model chooses to call a function, you will find tool_calls in the response. Extract the function call details and execute the corresponding function with the provided parameters.

In [7]:
tool_calls = completion.choices[0].message.tool_calls

if tool_calls: # Verify it's not empty
    # Extraction the function call details
    function_call_name = tool_calls[0].function.name # Get function name
    args = json.loads(tool_calls[0].function.arguments) # Get input arguments

    # Execute the corresponding function (NOTE: this is not done by the model)
    if function_call_name == "get_time":
        result = get_time(args["kind"])
        print(result)

Current date: 11/16/2025, Current time: 10:24:40


Once you have computed the result, pass it back to the model to get the final answer

In [8]:
if tool_calls:
    messages.append(completion.choices[0].message)  # Append the model's tool call message
    messages.append({                               # Append the result of executing the tool
        "role": "tool",
        "tool_call_id": tool_calls[0].id,
        "content": str(result)
    })
    display(messages)

    # Request the final response from the model
    completion_2 = client.chat.completions.create(
        model=MODEL,
        messages=messages,
    
    )
    print(completion_2.choices[0].message.content)

[{'role': 'system',
  'content': 'You are a helpful assistant with access to the get_time tool. Use the tool to get the time or date when needed, otherwise responds conversationally.'},
 {'role': 'user', 'content': 'What is the time and date now?'},
 ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_2799bf3648024ecc9f', function=ChoiceMessageToolCallFunction(arguments='{"kind":"both"}', name='get_time'), type='function', index=None)]),
 {'role': 'tool',
  'tool_call_id': 'call_2799bf3648024ecc9f',
  'content': 'Current date: 11/16/2025, Current time: 10:24:40'}]

The current time and date are 10:24:40 on 11/16/2025.
